<a href="https://colab.research.google.com/github/jaruizes/AI-CVMatcher/blob/main/ApplicationsScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scoring de aplicaciones a posiciones

Este notebook muestra como aplicar un scoring de un currículum aplicado a una posición


Se parte de datos no estructurados, como el currículúm vitae de una persona y se compara con los datos estructurados de una posición. La posición tiene el siguiente formato:

- descripción
- lista de requisitos: cada requisito tiene el formato <skill, nivel, descripción, obligatorio (sí/no)>
- lista de tareas a realizar
- tags de la oferta


Para transformar un CV a datos estructurados se utiliza un prompt, de modo que un LLM procese el CV y genere la siguiente estructura:

- resumen
- puntos clave del candidato
- puntos débiles del candidato
- hard skills
- soft skills
- tareas y responsabilidades que ha realizado en sus últimos 5 años
- posibles preguntas de la entrevista
- años totales de experiencia
- permanencia media en las empresas
- tags

El prompt para obtener esta estructura es el siguiente:



>
            You are an expert recruiter in technology profiles. You have to analyse and evaluate the resume text and to extract the following structured information:
            
             - **Summary** (max 400 words).
             - **Key Points** relevant about the candidate (education, career, skills, capabillites, interest, etc..).
             - **Weak Points** related to the whole document (education, career, skills, etc..)..
             - **Hard Skills** (For instance: Java, Python, AWS, GCP, Networks,....etc).
             - **Soft Skills** (For instance: Communication, Teamwork, Leadership, Problem-solving, etc...).
             - **Tasks and responsibilities during the last five years** (list of max 8 items evaluating the last five years worked).
             - **Potential Interview Questions about CV, skills, companies, permanency, roles and tasks, etc...**.
             - **Total Years of Experience**.
             - **Average permanence in years within the whole professional experiences**,
             - **Tags** (relevant tags about the candidate. For instance: Microservices, Architecture, Leadership, Java Expert, Good communicator,....)

            Format the output as a JSON object with the following structure:
            {
                "summary": "A summary of the resume",
                "keyPoints": ["key point 1", "key point 2", "key point 3"],
                "weakPoints": ["weak point 1", "weak point 2", "weak point 3"],
                "hardSkills": ["hard skill 1", "hardskill 2", "hardskill 3"],
                "softSkills": ["soft skill 1", "soft skill 2", "soft skill 3"],
                "tasksAndResponsabilities": ["Task and Responsibility 1", "Task and Responsibility 2", "Task and Responsibility 3"],
                "interviewQuestions": ["question 1", "question 2", "question 3"],
                "totalYearsExperience": 5,
                "averagePermanency": 2.5,
                "tags": ["tag 1", "tag 2", "tag 3"]
            }

            Resume Text:
            %s



## Librerías/Módulos necesarios

Importamos las librerías necesarias

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from google.colab import userdata
import openai
import time

## Modelo de OpenAI

Debemos configurar un **secreto OPENAI_KEY** que contenga la key que nos permite acceder al API de OpenAI. Para ello, simplemente pulsamos en el icono de la llave y creamos un nuevo secreto.

In [4]:
openai.api_key = userdata.get('OPENAI_KEY')

def get_embedding_openai(text):
    response = openai.embeddings.create(input=text, model="text-embedding-3-small")
    return np.array(response.data[0].embedding).reshape(1, -1)


## Mdelo LaBSE

Configuramos el modelo LaBSE (Language-Agnostic BERT Sentence Embeddings), Google Research que permite representar frases en un espacio vectorial, manteniendo su significado independientemente del idioma.

In [5]:
model_labse = SentenceTransformer("sentence-transformers/LaBSE")

def get_embedding_labse(text):
    return model_labse.encode([text])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

## Similitud semántica
A la hora de comparar los datos entre el currículum y la posición no queremos hacer una simple comparación de igualdad de términos o palabras sino que nos interesa una comparación semántica ya que cada persona puede exponer sus habilidades y experiencia de una forma diferente y no estructurada.

Definimos una función, que mediante la similitud del coseno, determina cómo de "parecidos" son los terminos (los embeddings que hemos generado) que se comparan

In [6]:
def compute_semantic_similarity(text1, text2, embedding_func):
    embedding1 = embedding_func(text1)
    embedding2 = embedding_func(text2)
    score = cosine_similarity(embedding1, embedding2)[0][0]
    return score

## Comparando requisitos

Definimos la función para realizar el score de los requisitos. Buscamos cada requisito de la posición en los skills del candidato, de forma semántica, y nos quedamos con el "best match".

Además, si el requisito es obligatorio tiene un peso mayor que si no lo es


In [7]:
def score_requirements(requirements, cv_skills, embedding_func):
    total_score = 0
    max_score = 0

    for req in requirements:
        skill = req["skill"]
        level = req["level"]
        desc = req["desc"]
        mandatory = req["mandatory"]

        best_match = max((compute_semantic_similarity(skill, cv_skill, embedding_func) for cv_skill in cv_skills), default=0)
        weight = 2 if mandatory.lower() == 'true' else 1
        total_score += best_match * weight
        max_score += weight

    return total_score / max_score if max_score > 0 else 0

## Comparando tareas a realizar

Definimos la función para calcular el score en el apartado de tareas. Como hemos indicado al inicio, con la llamada al LLM hemos obtenido las principales funciones que el candidato ha realizado en los últimos 5 años.

Realizamos una comparación similar a la de los requisitos. De forma semántica, obtenemos la distancia entre las tareas de la oferta y las tareas del candidato

In [8]:
def score_tasks(tasks, cv_tasks, embedding_func):
    cv_tasks = [task.lower() for task in cv_tasks]
    tasks = [task.lower() for task in tasks]
    if not tasks:
        return 0
    total_score = 0
    for task in tasks:
        best_match = max((compute_semantic_similarity(task, cv_task, embedding_func) for cv_task in cv_tasks), default=0)

        total_score += best_match

    return total_score / len(tasks)


## Cálculo del scoring

Por último, definimos la función de cálculo del scoring. El algoritmo que hemos elegido para este cálculo es el siguiente:

- calculamos la similitud entre los puntos fuertes del candidato y la descripción de la oferta. A este cálculo le asignaremos el menor valor de 0.1 ya que, a priori, es el que menos precisión puede tener
- calculamos la similitud entre los requisitos y los skills del candidato(agrupando tanto los hard como lo soft). Esta métrica va a ser la principal del scoring, con un valor de 0.6
- calculamos la similitud entre las tareas de la oferta y las que ha realizado el candidato en los últimos 5 años. Esta métrica tendrá un peso de 0.3

El valor final lo redondeamos, sin decimales, porque no tiene sentido que un candidato sea, por ejemplo, 70,68% compatible con una posición.

In [36]:
def compute_cv_score(id, offer, cv, embedding_func):
    print(id)
    start_time = time.time()

    cv_skills = cv['hardSkills'] + cv['softSkills'] + cv['tags']

    print('\t - Partial scores: ')
    desc_score = round(compute_semantic_similarity(offer['description'], str(cv['keyPoints']), embedding_func), 2)
    print(f'\t\t - desc_score: {desc_score}')

    requirement_score = round(score_requirements(offer['requirements'], cv_skills, embedding_func), 2)
    print(f'\t\t - requirement_score: {requirement_score}')

    task_score = round(score_tasks(offer['tasksAndResponsabilities'], cv['tasksAndResponsabilities'], embedding_func), 2)
    print(f'\t\t - task_score: {task_score}')

    final_score = round((0.1 * desc_score) + (0.6 * requirement_score) + (0.3 * task_score), 2)

    time_spent = round(time.time() - start_time, 2)


    print(f'\t - Final Score: {final_score}')
    # print(f'\t - Partial scores:  [desc_score: {desc_score}], [requirement_score: {requirement_score}], [task_score: {task_score}]')
    print(f'\t - Time spent: {time_spent} seconds')
    print('\n\n')

    return {
        'id': id,
        'score': final_score,
        'desc_score': desc_score,
        'requirement_score': requirement_score,
        'task_score': task_score,
        'time_spent': time_spent
    }

## Probando el algoritmo de scoring

Para probar el algoritmo vamos a realizar esta serie de pruebas usando el modelo de embeddings de OpenAI y LaBSE

### Datos de prueba
Para probar el scoring definimos dos posiciones y dos currículums:

* Posiciones:


In [2]:
offer_solutions_arq = {
  "description" : "Buscamos un arquitecto de soluciones con experiencia en diseño de arquitecturas escalables en la nube y microservicios.",
  "requirements" : [
    {"skill": "Software Architecture", "level": "experto", "desc": "Experiencia en diseño de arquitecturas escalables y resilientes.", "mandatory": "true"},
    {"skill": "Microservices", "level": "experto", "desc": "Implementación de arquitecturas basadas en microservicios.", "mandatory": "true"},
    {"skill": "Cloud", "level": "experto", "desc": "Experiencia con AWS, Azure o GCP", "mandatory": "true"},
    {"skill": "Kubernetes", "level": "medio", "desc": "Despliegue y gestión de aplicaciones en contenedores.", "mandatory": "true"},
    {"skill": "API", "level": "inicial", "desc": "Diseño de APIs REST y GraphQL.", "mandatory": "false"}
  ],
  "tasksAndResponsabilities": ["design scalable and high-performance architectures in the cloud",
                               "Asesorar a equipos de desarrollo en mejores prácticas de software",
                               "Diseñar e implementar estrategias de integración entre servicios",
                               "Garantizar la seguridad y gobernanza de las aplicaciones empresariales"],
  "tags": ["Architecture", "Cloud", "Microservices", "AWS", "Kubernetes", "Software Design"]
}

offer_front_dev = {
  "description" : "Buscamos un desarrollador frontend con experiencia en React y TypeScript para construir interfaces modernas y escalables.",
  "requirements" : [
    {"skill": "React", "level": "experto", "desc": "Experiencia en desarrollo con React y gestión de estado (Redux, Context API).", "mandatory": "true"},
    {"skill": "Typescript", "level": "experto", "desc": "Uso de TypeScript para mejorar la robustez del código.", "mandatory": "true"},
    {"skill": "CSS/SASS", "level": "experto", "desc": "Manejo de estilos avanzados y diseño responsivo.", "mandatory": "true"},
    {"skill": "UI/UX", "level": "inicial", "desc": "Conocimiento en diseño centrado en el usuario.", "mandatory": "false"},
    {"skill": "Testing", "level": "inicial", "desc": "Familiaridad con pruebas unitarias (Jest, React Testing Library).", "mandatory": "false"}
  ],
  "tasksAndResponsabilities": ["Desarrollar interfaces modernas y escalables con React y TypeScript.",
               "Colaborar con diseñadores para mejorar la experiencia de usuario.",
               "Optimizar el rendimiento y accesibilidad de la aplicación.",
               "Implementar pruebas unitarias para asegurar la calidad del código."],
  "tags": ["Frontend", "React", "TypeScript", "UI/UX", "JavaScript", "Web Development"]
}



* Currículums

In [3]:
cv_ai_expert = {
    "summary": "Luis Martínez García is a seasoned professional with over 12 years of experience in developing and implementing artificial intelligence solutions, with a recent focus on Generative AI. He is a recognized speaker at international events on AI, emphasizing how these technologies are transforming industries. Passionate about research and practical applications of advanced AI models, his goal is to lead innovative projects that promote the responsible adoption of artificial intelligence in businesses and organizations. His expertise includes machine learning and deep learning model design, along with proficiency in frameworks such as TensorFlow, PyTorch, and Hugging Face. Additionally, he has experience in cloud data management and MLOps, and is knowledgeable about AI ethics and regulation.",
    "keyPoints": [
        "Over 12 years of experience in AI solutions development.",
        "Expertise in Generative AI and NLP.",
        "Recognized speaker at international AI conferences.",
        "Strong background in machine learning and deep learning.",
        "Experience in MLOps and cloud environments."
    ],
    "weakPoints": [
        "Limited experience outside AI and machine learning.",
        "No formal management experience mentioned.",
        "Potential overemphasis on Generative AI despite broad AI experience.",
        "Focus primarily on technical skills with less emphasis on soft skills.",
        "May require upskilling in emerging AI regulations."
    ],
    "hardSkills": [
        "Machine Learning",
        "Deep Learning",
        "Generative AI",
        "Natural Language Processing (NLP)",
        "MLOps"
    ],
    "softSkills": [
        "Communication",
        "Leadership",
        "Teamwork",
        "Problem-solving",
        "Research"
    ],
    "tasksAndResponsabilities": [
        "Lead development projects based on Generative AI for various sectors.",
        "Create custom models for text and image generation.",
        "Implement AI systems in production platforms using Kubernetes and Docker.",
        "Develop advanced machine learning algorithms for predictive analysis.",
        "Research NLP models and contribute to scientific publications.",
        "Supervise research projects in collaboration with universities.",
        "Design and maintain data pipelines for machine learning projects.",
        "Train models for classification and data analysis."
    ],
    "interviewQuestions": [
        "Can you describe a successful project you led in Generative AI?",
        "How do you approach the ethical considerations in AI?",
        "What challenges have you faced when deploying AI solutions in production?",
        "How do you keep your skills updated in the rapidly evolving AI landscape?",
        "Can you provide an example of how you've collaborated with cross-functional teams?"
    ],
    "totalYearsExperience": 12,
    "tags": [
        "Artificial Intelligence",
        "Generative AI",
        "Machine Learning",
        "Deep Learning",
        "NLP",
        "MLOps"
    ]
}

cv_solutions_arq = {
    "summary": "Jose A. is a passionate technology expert with over 20 years of experience in strategic and technical roles across various sectors. He possesses extensive knowledge in Frontend, Backend, Cloud, APIs, and Databases, enabling him to manage complex architectures in high-performance enterprise environments. He emphasizes an end-to-end vision in solution delivery, ensuring that solutions are not only current but also scalable and adaptable for future needs. With experience in mentoring less experienced professionals, he actively contributes to the tech community through articles and presentations at events like Commit Conf and OpenExpo. His role as a Principal Solutions Architect at Paradigma Digital involves participation in pre-sales, designing end-to-end architectures, developing PoCs for new technologies, and standardizing best practices across the organization. He has led significant projects such as the digital transformation of document management for Mercadona and the regulatory adaptation for BME. His technical expertise spans a wide array of technologies, including Angular, Spring, Kubernetes, AWS, and GCP, and he holds multiple relevant certifications. José is committed to continuous learning and sharing knowledge, reflected in his publications and talks on modern architecture.",
    "keyPoints": [
        "Over 20 years of experience in technology roles.",
        "Expertise in end-to-end solution architecture.",
        "Strong mentoring and leadership skills.",
        "Active contributor to the tech community through articles and presentations.",
        "Proficient in a wide range of modern technologies and methodologies."
    ],
    "weakPoints": [
        "Limited recent experience in purely software development roles.",
        "Heavy focus on architecture may reduce hands-on coding experience.",
        "May require adaptation to rapidly changing technology trends.",
        "Potential gaps in experience with very niche technologies.",
        "Less emphasis on purely business-oriented roles."
    ],
    "hardSkills": [
        "Angular",
        "Spring",
        "Kubernetes",
        "Cloud",
        "API",
        "AWS",
        "Software Architecture",
        "Kubernetes",
        "Terraform",
        "Kafka",
        "Java",
        "Microservices",
        "CI/CD"
    ],
    "softSkills": [
        "Communication",
        "Leadership",
        "Mentoring",
        "Problem-solving",
        "Team collaboration"
    ],
    "tasksAndResponsabilities": [
        "Lead architectural design and development of end-to-end solutions.",
        "Conduct pre-sales assessments and client engagements.",
        "Develop and standardize best practices for development across the organization.",
        "Mentor junior staff and support their professional growth.",
        "Participate in technical events and publish related articles.",
        "Manage technical aspects of significant projects from inception to execution.",
        "Design and develop Proofs of Concept (PoCs) for new technologies.",
        "Oversee the technical implementation and provide functional support."
    ],
    "interviewQuestions": [
        "Can you describe your approach to developing end-to-end architectures?",
        "What strategies do you use for mentoring junior team members?",
        "How do you keep up with new technologies and trends in the industry?",
        "Can you discuss a challenging project you managed and how you overcame obstacles?",
        "What role do you believe documentation and standardization play in your projects?"
    ],
    "totalYearsExperience": 20,
    "averagePermanency": 2.5,
    "tags": [
        "Microservices",
        "Architecture",
        "Leadership",
        "Java Expert",
        "Cloud Solutions"
    ]
}

### Pruebas Resultados

Se van a realizar las siguientes pruebas con las siguientes expectativas
- scoring de la posición de arquitecto de soluciones con el cv del arquitecto de soluciones --> debe dar un scoring alto, por encima del 65%
- scoring de la posición de desarrollador front con el cv de experto en IA --> debe dar un scoring bajo, por debajo del 40%
- scoring de la posición de desarrollador front con el cv de arquitecto --> debería dar un scoring bajo pero mayor que el caso anterior y menor que el caso inicial

In [37]:
## OpenAI tests
result = compute_cv_score('[OpenAI] Pos: Solutions Arch -> CV: Solutions Arch', offer_solutions_arq, cv_solutions_arq, get_embedding_openai)
assert result['score'] > 0.65

result = compute_cv_score('[OpenAI] Pos: Front Dev -> CV: AI Expert', offer_front_dev, cv_ai_expert, get_embedding_openai)
assert result['score'] < 0.40

result = compute_cv_score('[OpenAI] Pos: Front Dev -> CV: Solutions Arch', offer_front_dev, cv_solutions_arq, get_embedding_openai)
assert result['score'] > 0.40 and result['score'] < 0.65

## LaBSE tests
result = compute_cv_score('[LaBSE] Pos: Solutions Arch -> CV: Solutions Arch', offer_solutions_arq, cv_solutions_arq, get_embedding_labse)
assert result['score'] > 0.65

result = compute_cv_score('[LaBSE] Pos: Front Dev -> CV: AI Expert', offer_front_dev, cv_ai_expert, get_embedding_openai)
assert result['score'] < 0.40

result = compute_cv_score('[LaBSE] Pos: Front Dev -> CV: Solutions Arch', offer_front_dev, cv_solutions_arq, get_embedding_labse)
assert result['score'] > 0.40 and result['score'] < 0.65



[OpenAI] Pos: Solutions Arch -> CV: Solutions Arch
	 - Partial scores: 
		 - desc_score: 0.33
		 - requirement_score: 1.0
		 - task_score: 0.45
	 - Final Score: 0.77
	 - Time spent: 79.68 seconds



[OpenAI] Pos: Front Dev -> CV: AI Expert
	 - Partial scores: 
		 - desc_score: 0.17
		 - requirement_score: 0.3
		 - task_score: 0.31
	 - Final Score: 0.29
	 - Time spent: 46.26 seconds



[OpenAI] Pos: Front Dev -> CV: Solutions Arch
	 - Partial scores: 
		 - desc_score: 0.28
		 - requirement_score: 0.47
		 - task_score: 0.36
	 - Final Score: 0.42
	 - Time spent: 65.04 seconds



[LaBSE] Pos: Solutions Arch -> CV: Solutions Arch
	 - Partial scores: 
		 - desc_score: 0.47999998927116394
		 - requirement_score: 1.0
		 - task_score: 0.56
	 - Final Score: 0.82
	 - Time spent: 26.74 seconds



[LaBSE] Pos: Front Dev -> CV: AI Expert
	 - Partial scores: 
		 - desc_score: 0.17
		 - requirement_score: 0.3
		 - task_score: 0.31
	 - Final Score: 0.29
	 - Time spent: 47.34 seconds



[LaBSE] Pos: Fro